### Data Cleaning for KAP Surveys - Bringing in Demographic Data

##### Bringing Demographic data into account for combined KAP Survey #1 and #2 results and creating a social desirability index
##### Bringing SGD data and created social desirability index together 

###### 15/07/2021 at 04:05 CEST

In [1]:
# import necessary libraries
import os
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
plt.style.use('seaborn-pastel')

In [2]:
# change working directory to a specified directory
os.chdir('../')
print("Directory Changes")

# Get current working directory
cwd = os.getcwd()
print("Current working directory is:", cwd)

Directory Changes
Current working directory is: C:\Users\Carol\Documents\EPA_2020_2021\Thesis\Analysis\KAP_Analysis


In [5]:
# read in csv data
kap12 = pd.read_csv("data/interim/kap12_clean.csv")
sd = pd.read_csv("data/raw/SocialDesirabilitySurvey_Total.csv")
sgd= pd.read_csv("data/interim/sgd_clean.csv")

## Prepare Demographic Data

In [6]:
# gain insight into sd
sd.shape

(51, 35)

In [7]:
# gain insight into kap1
sd.head(5)

,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,...,Q9,Q10,Q11,Q12,Q13,Q14,Q15,Q16,Q17,Random ID
0,Start Date,End Date,Response Type,IP Address,Progress,Duration (in seconds),Finished,Recorded Date,Response ID,Recipient Last Name,...,"I can remember ""playing sick"" to get out of so...",There have been occasions when I have taken ad...,I sometimes try to get even rather than forgiv...,"I am always courteous, even to people who are ...",There have been occasions when I felt like sma...,I have never been irked when people expressed ...,There have been times when I was quite jealous...,I sometimes think when people have a misfortun...,I have never deliberately said something that ...,Random ID
1,"{""ImportId"":""startDate"",""timeZone"":""America/Lo...","{""ImportId"":""endDate"",""timeZone"":""America/Los_...","{""ImportId"":""status""}","{""ImportId"":""ipAddress""}","{""ImportId"":""progress""}","{""ImportId"":""duration""}","{""ImportId"":""finished""}","{""ImportId"":""recordedDate"",""timeZone"":""America...","{""ImportId"":""_recordId""}","{""ImportId"":""recipientLastName""}",...,"{""ImportId"":""QID15""}","{""ImportId"":""QID16""}","{""ImportId"":""QID17""}","{""ImportId"":""QID18""}","{""ImportId"":""QID19""}","{""ImportId"":""QID20""}","{""ImportId"":""QID21""}","{""ImportId"":""QID22""}","{""ImportId"":""QID23""}","{""ImportId"":""Random ID""}"
2,03/08/2021 13:34,03/08/2021 13:36,IP Address,68.195.20.120,100,128,TRUE,03/08/2021 13:36,R_2fm8SJ2dHpD1WpJ,NaN,...,TRUE,TRUE,FALSE,TRUE,TRUE,FALSE,TRUE,FALSE,FALSE,3421154104
3,03/08/2021 13:35,03/08/2021 13:36,IP Address,73.9.31.16,100,76,TRUE,03/08/2021 13:36,R_1kUHbiBmgUJMbUY,NaN,...,TRUE,TRUE,TRUE,TRUE,FALSE,FALSE,TRUE,TRUE,TRUE,9292685410
4,03/08/2021 13:35,03/08/2021 13:36,IP Address,70.59.74.66,100,94,TRUE,03/08/2021 13:36,R_2EmhSufCcUNXuuX,NaN,...,TRUE,TRUE,TRUE,FALSE,TRUE,FALSE,TRUE,TRUE,FALSE,3955286470


In [8]:
# get names of columns
print(sd.columns)

Index(['StartDate', 'EndDate', 'Status', 'IPAddress', 'Progress',
       'Duration (in seconds)', 'Finished', 'RecordedDate', 'ResponseId',
       'RecipientLastName', 'RecipientFirstName', 'RecipientEmail',
       'ExternalReference', 'LocationLatitude', 'LocationLongitude',
       'DistributionChannel', 'UserLanguage', 'Q1', 'Q2', 'Q3', 'Q4', 'Q5',
       'Q6', 'Q7', 'Q8', 'Q9', 'Q10', 'Q11', 'Q12', 'Q13', 'Q14', 'Q15', 'Q16',
       'Q17', 'Random ID'],
      dtype='object')


In [9]:
# drop unnecessary unnecessary columns in sd
sd = sd.drop(['Progress','StartDate','EndDate', 'Status', 'IPAddress', 'Duration (in seconds)', 'Finished', 'RecordedDate', 'ResponseId',
                  'RecipientLastName', 'RecipientFirstName', 'RecipientEmail', 'ExternalReference', 'LocationLatitude', 'LocationLongitude',
                 'DistributionChannel', 'UserLanguage'], axis = 1)


In [10]:
# drop unnecessary rows
sd.dropna(subset = ["Random ID"], inplace = True)
sd = sd.iloc[2:]

In [12]:
# take only rows with non Nan values?
# df = df[df['EPS'].notna()]
sd = sd[sd['Q1'].notna()==True]

In [13]:
# drop duplicates
sd = sd.drop_duplicates(subset= 'Random ID', keep='first', inplace=False)

In [14]:
# reset index to "Random ID"
sd.set_index(["Random ID"], inplace = True, append = True, drop = True)
sd.reset_index(inplace = True)
sd.head(10)

,level_0,Random ID,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,Q11,Q12,Q13,Q14,Q15,Q16,Q17
0,2,3421154104,Female,25 - 34,White or Caucasian,Community member,FALSE,TRUE,TRUE,TRUE,TRUE,TRUE,FALSE,TRUE,TRUE,FALSE,TRUE,FALSE,FALSE
1,3,9292685410,Female,18 - 24,Asian or Pacific Islander,Community member,FALSE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,FALSE,FALSE,TRUE,TRUE,TRUE
2,4,3955286470,Male,35 - 44,White or Caucasian,Community member,FALSE,TRUE,FALSE,FALSE,TRUE,TRUE,TRUE,FALSE,TRUE,FALSE,TRUE,TRUE,FALSE
3,5,8719080354,Male,35 - 44,Black or African American,Law enforcement officer OR Former law enforcem...,TRUE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,TRUE,FALSE,FALSE,FALSE
4,6,4855207987,Female,35 - 44,Asian or Pacific Islander,Community member,FALSE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,FALSE,TRUE,FALSE,TRUE,TRUE,FALSE
5,7,9720007520,Male,25 - 34,White or Caucasian,Community member,TRUE,TRUE,TRUE,TRUE,FALSE,TRUE,TRUE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE
6,8,9237049872,Female,25 - 34,Black or African American,Community member,FALSE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,FALSE,TRUE,FALSE,FALSE
7,9,1878269020,Male,25 - 34,Black or African American,Community member,FALSE,FALSE,TRUE,TRUE,TRUE,FALSE,FALSE,TRUE,TRUE,FALSE,FALSE,FALSE,FALSE
8,10,7304834385,Female,25 - 34,Asian or Pacific Islander,Community member,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,TRUE,TRUE,TRUE,FALSE,TRUE,FALSE,TRUE
9,11,8645952153,Female,25 - 34,Black or African American,Community member,FALSE,TRUE,TRUE,FALSE,TRUE,FALSE,FALSE,TRUE,TRUE,FALSE,FALSE,FALSE,FALSE


In [15]:
# drop "level_0" column
sd = sd.drop(['level_0'], axis = 1)

### Recode True/False SD Answers

Recode True/False Answers to 1/0 depending on if something scores "yes" (i.e. 1) to socially desirable or "no" (i.e. 0) to socially desirable



Where the answer key is as follows:

Q5: True
Q6: False
Q7: False
Q8: False
Q9: False
Q10: False
Q11: False
Q12: True
Q13: False
Q14: True
Q15: False
Q16: False
Q17: True

In [16]:
# recode True/False Questions to 1 for correct (i.e. correct for high social desirability) answers and 0 for wrong answers (i.e. correct for high social desirability)

# Q5
sd.loc[sd.Q5 == 'TRUE', 'Q5'] = 1
sd.loc[sd.Q5 == 'FALSE', 'Q5'] = 0

# Q6
sd.loc[sd.Q6 == 'TRUE', 'Q6'] = 0
sd.loc[sd.Q6 == 'FALSE', 'Q6'] = 1

# Q7
sd.loc[sd.Q7 == 'TRUE', 'Q7'] = 0
sd.loc[sd.Q7 == 'FALSE', 'Q7'] = 1

# Q8
sd.loc[sd.Q8 == 'TRUE', 'Q8'] = 0
sd.loc[sd.Q8 == 'FALSE', 'Q8'] = 1

# Q9
sd.loc[sd.Q9 == 'TRUE', 'Q9'] = 0
sd.loc[sd.Q9 == 'FALSE', 'Q9'] = 1

# Q10
sd.loc[sd.Q10 == 'TRUE', 'Q10'] = 0
sd.loc[sd.Q10 == 'FALSE', 'Q10'] = 1

# Q11
sd.loc[sd.Q11 == 'TRUE', 'Q11'] = 0
sd.loc[sd.Q11 == 'FALSE', 'Q11'] = 1

# Q12
sd.loc[sd.Q12 == 'TRUE', 'Q12'] = 1
sd.loc[sd.Q12 == 'FALSE', 'Q12'] = 0

# Q13
sd.loc[sd.Q13 == 'TRUE', 'Q13'] = 0
sd.loc[sd.Q13 == 'FALSE', 'Q13'] = 1

# Q14
sd.loc[sd.Q14 == 'TRUE', 'Q14'] = 1
sd.loc[sd.Q14 == 'FALSE', 'Q14'] = 0

# Q15
sd.loc[sd.Q15 == 'TRUE', 'Q15'] = 0
sd.loc[sd.Q15 == 'FALSE', 'Q15'] = 1

# Q16
sd.loc[sd.Q16 == 'TRUE', 'Q16'] = 0
sd.loc[sd.Q16 == 'FALSE', 'Q16'] = 1

# Q17
sd.loc[sd.Q17 == 'TRUE', 'Q17'] = 1
sd.loc[sd.Q17 == 'FALSE', 'Q17'] = 0

In [18]:
# create social desirability score by adding columns Q5, Q6, Q7, Q8, Q9, Q10, Q11, Q12, Q13, Q14, Q15, Q16, and Q17, 
# divding by 13 questions, and subtracting this value from 1
# sd['sd_score'] = 1-((sd['Q5'] + sd['Q6'] + sd['Q7'] + sd['Q8'] + sd['Q9'] + sd['Q10'] + 
#                       sd['Q11'] + sd['Q12'] + sd['Q13'] + sd['Q14'] + sd['Q15'] + sd['Q16'] + sd['Q17'])/13)

sd['sd_score'] = 1 - (sd[['Q5', 'Q6', 'Q7', 'Q8', 'Q9', 'Q10', 'Q11', 'Q12', 'Q13', 'Q14', 'Q15', 'Q16', 'Q17']].mean(axis=1))

In [19]:
# gain insight into sd_copy

# where a high sd_score indicates that your answers for attitudes and practices can be taken at more or less face value
# and where a low sd_score indicates that your answers for attitudes and practices cannot be taken more or less at face value 
sd.head()

,Random ID,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,Q11,Q12,Q13,Q14,Q15,Q16,Q17,sd_score
0,3421154104,Female,25 - 34,White or Caucasian,Community member,0,0,0,0,0,0,1,1,0,0,0,1,0,0.769231
1,9292685410,Female,18 - 24,Asian or Pacific Islander,Community member,0,0,0,0,0,0,0,1,1,0,0,0,1,0.769231
2,3955286470,Male,35 - 44,White or Caucasian,Community member,0,0,1,1,0,0,0,0,0,0,0,0,0,0.846154
3,8719080354,Male,35 - 44,Black or African American,Law enforcement officer OR Former law enforcem...,1,1,0,1,1,1,1,1,1,1,1,1,0,0.153846
4,4855207987,Female,35 - 44,Asian or Pacific Islander,Community member,0,0,0,0,0,0,0,0,0,0,0,0,0,1.000000


In [20]:
# drop unnecessary unnecessary columns in sd of Q5, Q6, Q7, Q8, Q9, Q10, Q11, Q12, Q13, Q14, Q15, Q16, and Q17
sd = sd.drop(['Q5', 'Q6', 'Q7', 'Q8', 'Q9', 'Q10', 'Q11', 'Q12', 'Q13', 'Q14', 'Q15', 'Q16', 'Q17'], axis = 1)

In [21]:
# rename Q1 (gender), Q2 (age), Q3 (race), and Q4 (profession) columns
sd.columns = ['RandomID', 'gender', 'age', 'race', 'profession', 'sd_score']

## Bring SD and KAP data together

In [22]:
# prepare "RandomID" column keys for merging
sd['RandomID'] = sd['RandomID'].astype(str)
kap12['RandomID'] = kap12['RandomID'].astype(str)

In [23]:
# merge sd data and kap12 data on "Random ID"
kap_data = pd.merge(kap12, sd, on='RandomID', how='outer')

In [25]:
# gain insight into kap_data
kap_data.head(50)

,RandomID,pre mean a_pc,pre mean a_er,pre mean p_pc,pre mean k_pc,Q*,post mean a_pc,post mean a_er,post mean p_pc,post mean k_pc,diff_a_pc,diff_a_er,diff_p_pc,diff_k_pc,protocol,gender,age,race,profession,sd_score
0,3955286470,1.333333,4.333333,3.666667,1.000000,B4,1.666667,4.000000,3.333333,1.000000,0.333333,-0.333333,-0.333333,0.000000,B,Male,35 - 44,White or Caucasian,Community member,0.846154
1,3421154104,1.000000,4.333333,4.000000,1.000000,B5,1.666667,4.333333,4.333333,1.000000,0.666667,0.000000,0.333333,0.000000,B,Female,25 - 34,White or Caucasian,Community member,0.769231
2,1878269020,1.333333,4.666667,3.333333,1.000000,B2,1.333333,4.666667,3.333333,1.000000,0.000000,0.000000,0.000000,0.000000,B,Male,25 - 34,Black or African American,Community member,0.538462
3,4855207987,3.000000,5.000000,2.666667,0.666667,B2,2.666667,5.000000,3.000000,1.000000,-0.333333,0.000000,0.333333,0.333333,B,Female,35 - 44,Asian or Pacific Islander,Community member,1.000000
4,6034387383,1.333333,4.333333,3.333333,1.000000,B1,1.333333,3.000000,3.333333,1.000000,0.000000,-1.333333,0.000000,0.000000,B,Female,25 - 34,White or Caucasian,Community member,0.846154
5,9237049872,2.000000,4.333333,3.666667,1.000000,B2,2.000000,4.666667,4.000000,1.000000,0.000000,0.333333,0.333333,0.000000,B,Female,25 - 34,Black or African American,Community member,0.846154
6,8645952153,2.000000,3.333333,4.000000,1.000000,B2,2.000000,3.666667,4.333333,1.000000,0.000000,0.333333,0.333333,0.000000,B,Female,25 - 34,Black or African American,Community member,0.538462
7,3413361853,2.000000,4.666667,4.333333,1.000000,B1,2.000000,4.666667,4.666667,1.000000,0.000000,0.000000,0.333333,0.000000,B,Female,18 - 24,White or Caucasian,Community member,0.538462
8,3448845540,2.333333,3.333333,3.333333,1.000000,B1,2.666667,3.333333,3.333333,1.000000,0.333333,0.000000,0.000000,0.000000,B,Female,25 - 34,Asian or Pacific Islander,Community member,0.538462
9,8185810064,2.333333,5.000000,5.000000,1.000000,B4,2.333333,5.000000,4.000000,1.000000,0.000000,0.000000,-1.000000,0.000000,B,Female,25 - 34,Black or African American,Community member,0.538462


In [26]:
# drop "participants" who did not fill out all KAP #1, KAP #2, and sd surveys
kap_data.dropna(subset = ["pre mean a_pc"], inplace = True)
kap_data.dropna(subset = ["pre mean a_er"], inplace = True)
kap_data.dropna(subset = ["pre mean p_pc"], inplace = True)
kap_data.dropna(subset = ["pre mean k_pc"], inplace = True)

kap_data.dropna(subset = ["post mean a_pc"], inplace = True)
kap_data.dropna(subset = ["post mean a_er"], inplace = True)
kap_data.dropna(subset = ["post mean p_pc"], inplace = True)
kap_data.dropna(subset = ["post mean k_pc"], inplace = True)

In [27]:
# gain insight into kap_data
kap_data.head(50)

,RandomID,pre mean a_pc,pre mean a_er,pre mean p_pc,pre mean k_pc,Q*,post mean a_pc,post mean a_er,post mean p_pc,post mean k_pc,diff_a_pc,diff_a_er,diff_p_pc,diff_k_pc,protocol,gender,age,race,profession,sd_score
0,3955286470,1.333333,4.333333,3.666667,1.000000,B4,1.666667,4.000000,3.333333,1.000000,0.333333,-0.333333,-0.333333,0.000000,B,Male,35 - 44,White or Caucasian,Community member,0.846154
1,3421154104,1.000000,4.333333,4.000000,1.000000,B5,1.666667,4.333333,4.333333,1.000000,0.666667,0.000000,0.333333,0.000000,B,Female,25 - 34,White or Caucasian,Community member,0.769231
2,1878269020,1.333333,4.666667,3.333333,1.000000,B2,1.333333,4.666667,3.333333,1.000000,0.000000,0.000000,0.000000,0.000000,B,Male,25 - 34,Black or African American,Community member,0.538462
3,4855207987,3.000000,5.000000,2.666667,0.666667,B2,2.666667,5.000000,3.000000,1.000000,-0.333333,0.000000,0.333333,0.333333,B,Female,35 - 44,Asian or Pacific Islander,Community member,1.000000
4,6034387383,1.333333,4.333333,3.333333,1.000000,B1,1.333333,3.000000,3.333333,1.000000,0.000000,-1.333333,0.000000,0.000000,B,Female,25 - 34,White or Caucasian,Community member,0.846154
5,9237049872,2.000000,4.333333,3.666667,1.000000,B2,2.000000,4.666667,4.000000,1.000000,0.000000,0.333333,0.333333,0.000000,B,Female,25 - 34,Black or African American,Community member,0.846154
6,8645952153,2.000000,3.333333,4.000000,1.000000,B2,2.000000,3.666667,4.333333,1.000000,0.000000,0.333333,0.333333,0.000000,B,Female,25 - 34,Black or African American,Community member,0.538462
7,3413361853,2.000000,4.666667,4.333333,1.000000,B1,2.000000,4.666667,4.666667,1.000000,0.000000,0.000000,0.333333,0.000000,B,Female,18 - 24,White or Caucasian,Community member,0.538462
8,3448845540,2.333333,3.333333,3.333333,1.000000,B1,2.666667,3.333333,3.333333,1.000000,0.333333,0.000000,0.000000,0.000000,B,Female,25 - 34,Asian or Pacific Islander,Community member,0.538462
9,8185810064,2.333333,5.000000,5.000000,1.000000,B4,2.333333,5.000000,4.000000,1.000000,0.000000,0.000000,-1.000000,0.000000,B,Female,25 - 34,Black or African American,Community member,0.538462


In [28]:
# write kap_data to csv
kap_data.to_csv('data/interim/kap_sd_clean.csv', encoding='utf-8', index=False)

## Bring SD and SGD data together

In [29]:
# gain insight into sgd
sgd.head()

,RandomID,Q*,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,Q11,total mean sgd_effect,mean sgd_sf,mean sgd_u,mean sgd_tc,protocol
0,8194933674,A1,5,5,5,5,5,5.0,5,5,5,5,5,5.000000,5.000000,5.0,5.00,A
1,9542137183,A1,5,5,4,5,4,5.0,4,5,4,5,5,4.636364,4.666667,4.5,4.75,A
2,8323282184,A1,5,4,5,5,5,4.0,4,5,5,5,5,4.727273,4.666667,4.5,5.00,A
3,4165424527,A1,5,5,5,5,5,5.0,5,5,5,5,5,5.000000,5.000000,5.0,5.00,A
4,3648895874,A2,3,4,5,3,3,4.0,4,4,4,3,3,3.636364,4.000000,3.5,3.50,A


In [30]:
# prepare "RandomID" column keys for merging
sd['RandomID'] = sd['RandomID'].astype(str)
sgd['RandomID'] = sgd['RandomID'].astype(str)

In [31]:
# merge sd data and sgd data on "Random ID"
sgd_data = pd.merge(sgd, sd, on='RandomID', how='outer')

In [33]:
# gain insight into sgd_data
sgd_data.head(50)

,RandomID,Q*,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,...,total mean sgd_effect,mean sgd_sf,mean sgd_u,mean sgd_tc,protocol,gender,age,race,profession,sd_score
0,8194933674,A1,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,...,5.000000,5.000000,5.000000,5.00,A,Male,18 - 24,White or Caucasian,Community member,0.384615
1,9542137183,A1,5.0,5.0,4.0,5.0,4.0,5.0,4.0,5.0,...,4.636364,4.666667,4.500000,4.75,A,Female,18 - 24,White or Caucasian,Community member,0.923077
2,8323282184,A1,5.0,4.0,5.0,5.0,5.0,4.0,4.0,5.0,...,4.727273,4.666667,4.500000,5.00,A,Male,18 - 24,White or Caucasian,Community member,0.615385
3,4165424527,A1,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,...,5.000000,5.000000,5.000000,5.00,A,Male,18 - 24,White or Caucasian,NaN,0.692308
4,3648895874,A2,3.0,4.0,5.0,3.0,3.0,4.0,4.0,4.0,...,3.636364,4.000000,3.500000,3.50,A,Male,25 - 34,White or Caucasian,Community member,0.461538
5,7414114848,A2,4.0,5.0,5.0,2.0,3.0,4.0,4.0,4.0,...,3.909091,4.666667,3.250000,4.00,A,Female,25 - 34,Asian or Pacific Islander,Community member,0.538462
6,8366140919,A2,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,4.000000,4.000000,4.000000,4.00,A,Female,25 - 34,White or Caucasian,Community member,0.384615
7,8864792088,A2,4.0,5.0,5.0,4.0,4.0,5.0,4.0,5.0,...,4.363636,4.666667,4.250000,4.25,A,Female,25 - 34,White or Caucasian,Community member,0.769231
8,3413361853,B1,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,...,4.818182,5.000000,5.000000,4.50,B,Female,18 - 24,White or Caucasian,Community member,0.538462
9,6034387383,B1,4.0,4.0,5.0,4.0,4.0,5.0,3.0,5.0,...,4.000000,4.333333,4.000000,3.75,B,Female,25 - 34,White or Caucasian,Community member,0.846154


In [34]:
# write sgd_data to csv
sgd_data.to_csv('data/interim/sgd_sd_clean.csv', encoding='utf-8', index=False)